In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
import pandas as pd

In [3]:
docs=pd.read_csv('Talent Snapshot - RAW.csv')
docs=docs.iloc[:,1:]
docs.columns=['Edu','Talent','Expectation','Single/Team','Remote','Startdate',
              'InterestedPlatform','Startup','Team','Howaddvalue','Achievement','Idealjob',
             'Other']
for i in range(len(docs)):
    if type(docs['Talent'][i])==str:
    #if not np.isnan(docs['Talent'][i]):
        docs.iloc[i,1]=' '.join(docs['Talent'][i].split('\xa0'))
#####Processing of column's name and the \xa0 in string

In [90]:
docs

,Edu,Talent,Expectation,Single/Team,Remote,Startdate,InterestedPlatform,Startup,Team,Howaddvalue,Achievement,Idealjob,Other
0,Santa Clara University,"More than technical skills, I would say am goo...",My prior experience was different from what I ...,Doesn't matter.,"If it’s an option, I don't mind working remotely.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"University of California, Riverside","In regards to my talents, I have a strong bac...",I hope to work in the data science field to p...,Working as a team or working by myself are bot...,Commuting to and from work is not an issue sin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"University of California, Berkeley",I have exceptional time management skills. As ...,Though I am relatively new to programming as I...,"Though I believe I am able to handle either, I...","No, I will not be working remotely.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,San Jose State University,I am goo with Back-end development and Machine...,I want to work on any kind of software develop...,"Doesn't Matter, but prefer working on team",I am fine with anything,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"University of California, San Diego",I'm talented at self-control. I would finish m...,I would like to know more people in the workpl...,I would prefer working individually on a small...,"Because I'm still studying in UC San Diego, I ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3305,I am highly passionate to solve complex proble...,I have my specialization in Data Science with ...,I was finalist in a team of 6 in Smart City Ha...,Ideally any place where my work reflects an im...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3306,I think my greatest strength is my strong lead...,I can extract massive data using Hive SQL and ...,I believe my biggest achievement is that I hav...,"In terms of work content, I hope I can work wi...",Thank you for asking! I have nothing more to s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3307,One of my main talents is UX design. My unders...,"For FORKAIA ACADEMY, I can help optimize socia...",I have received many awards for my GPA and cre...,My fantasy job would be working in digital mar...,I am extremely diligent and productive.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3308,I learn to adapt to jobs quickly using past ex...,I have experience cleaning data. I can analyze...,Graduating EOD training and being a fully func...,"Auditor: specifically, analyzing waste and log...",I am open to anything that will help me grow. ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [114]:
cv = CountVectorizer(stop_words='english',ngram_range=(1,2))
X = cv.fit_transform(docs['Talent'][docs['Talent'].notnull()])
word_dic={}
for i,j in cv.vocabulary_.items():
    word_dic[j]=i

In [115]:
a=np.array(X.todense().sum(axis=0))[0]
word_mapping=[]
for i,j in enumerate(a):
    word_mapping.append((i,j))

In [116]:
def max_word(cv,num=5):
    count=np.array(cv.todense().sum(axis=0))[0]
    word_mapping=[]
    for i,j in enumerate(count):
        word_mapping.append((i,j))
    return sorted(word_mapping,key=lambda x:x[1])[::-1][:num]
max_5=[i[0] for i in max_word(X,20)]
[word_dic[i] for i in max_5]

['data',
 'skills',
 'work',
 'learning',
 'experience',
 'good',
 'python',
 'working',
 'analysis',
 'team',
 'new',
 'time',
 'learn',
 'people',
 'business',
 'problem',
 'science',
 'programming',
 'project',
 'like']

In [127]:
def search_num(cv,X,l):
    a=np.array(X.todense().sum(axis=0))[0]
    result=0
    for i in l:
        if i in cv.vocabulary_:
            result+=a[cv.vocabulary_[i]]
    return result
search_num(cv,X,['mds'])

0

In [124]:
word_dic={'bachelor':['bachelor','ba','bs','bachelors'],
'master':['master','ma','ms','mba','masters'],
'phd':['phd','doctor'],
'data':['data scientist','data analyst','data engineering','data science'],
'cs':['computer science','computer scientist'],
'sde':['software engineering'],
'machine learning':['machine learning','deep learning']}

#a[cv.vocabulary_['competition']]

#word_dic_num

In [126]:
cv2 = CountVectorizer(stop_words='english',ngram_range=(1,2))
X_school = cv2.fit_transform(docs['Edu'][docs['Edu'].notnull()])
word_dic_num={}
word=['bachelor','master','phd','data','cs','sde','machine learning']
for i in word:
    word_dic_num[i]=search_num(cv,X,word_dic[i])
for i in word:
    word_dic_num[i]+=search_num(cv2,X_school,word_dic[i])
word_dic_num

{'bachelor': 1382,
 'master': 1324,
 'phd': 29,
 'data': 590,
 'cs': 731,
 'sde': 57,
 'machine learning': 659}

In [22]:
tfidf = TfidfVectorizer(ngram_range=(1,2),
                        stop_words='english',
                        norm=False,
                        smooth_idf=False
                       )
X2 = tfidf.fit_transform(docs['Talent'][docs['Talent'].notnull()])

In [29]:
vocabulary=tfidf.get_feature_names()

In [25]:
lda = LatentDirichletAllocation(n_components=20,# the number of topics
                                n_jobs=-1,        # use all cpus
                                random_state=123) # for reproducability
X_lda = lda.fit_transform(X2)

In [30]:
for i in range(20):
    words=np.array(vocabulary)[np.argsort(lda.components_[i])[::-1][:5]]
    print(f"Topoc #{i}:{' '.join(words)}")

Topoc #0:data learning skills experience analysis
Topoc #1:data skills people team good
Topoc #2:data good analysis skills python
Topoc #3:skills data work team learning
Topoc #4:data skills problem experience work
Topoc #5:data skills work people learning
Topoc #6:data skills good experience learning
Topoc #7:data skills work team marketing
Topoc #8:data skills science programming analysis
Topoc #9:data work new experience good
Topoc #10:data work skills python team
Topoc #11:data skills work research learning
Topoc #12:data skills work experience learning
Topoc #13:data skills design learning work
Topoc #14:data skills analysis work new
Topoc #15:data business skills work working
Topoc #16:skills data experience work learning
Topoc #17:data science skills experience learning
Topoc #18:data new work learning team
Topoc #19:data work skills business analysis
